In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sys import stdout

# EZMC: Easy-Peasy MCMC

EZMC provides a simple interface to Markov Chain Monte Carlo algorithms for Bayesian inference.

It is

- Simple. It's written in pure python, with few dependencies.
- Easy to use.
- Flexible. 
  - All you have to do is write a function that takes parameter values as inputs, and returns log posterior densities.
  - It also works with noisy functions, like when you need to simulate data from a computational model.
- Extendable. New MCMC algorithms can be implemented by overwriting methods (e.g. `propose()`, `eval_proposal()`) of generic ones.

It's also

- Slow. EZMC doesn't use the likes of Theano or TensorFlow as a backend. This means that it's much simpler to plug in your own posterior density functions, but also means it isn't nearly as fast as packages like [PyMC3]().
- A work in progress. There's lots of room for immprovement!

Check out [github.com/Gabriel-p/pythonMCMC](https://github.com/Gabriel-p/pythonMCMC) for a great list of other python MCMC samplers.



In [ ]:
## Reload stuff
from importlib import reload
import ezmc
import ezmc
reload(ezmc)


## Simple Example

Let's start with the simplest example.
Say the posterior distribution we wish to sample from is just a standard 2-d Gaussian:

$$
\theta \sim Normal(\mu, \Sigma);\\
\mu = \begin{bmatrix}0 & 0\end{bmatrix};
\Sigma = \begin{bmatrix}1 & 0 \\ 0 & 1\end{bmatrix};
$$

We can actually sample directly from this distribution, for comparision

In [ ]:
def sample_true(n):
    dist = stats.multivariate_normal(mean=[0, 0], cov=[[1,0], [0, 1]])
    return dist.rvs(n).T
true_posterior = sample_true(1000)
true_posterior.mean(1).round(2), true_posterior.std(1).round(2)

In [ ]:
def setup_axes(newfigure=True):
    if newfigure:
        fig = plt.figure(figsize=(4,4))
    plt.xlabel('$θ_1$')
    plt.ylabel('$θ_2$')
    plt.xlim(-5, 5)
    plt.ylim(-5, 5)

setup_axes()
plt.scatter(true_posterior[0], true_posterior[1], alpha=.5)

To sample from this distribution using EZMC, we need to write two functions.
The postererior density function taskes an array of parameters as an input 
(in this case, an array of length 2) 
and returns the log-likelihood as an output.
The initialisation function just returns random starting values for the sampler.

In [ ]:
def posterior_density(x):
    dist = stats.multivariate_normal(mean=[0, 0], cov=[[1,0], [0, 1]])
    return dist.logpdf(x)

## In this case, could also use
# def posterior_density(x):
#     a, b = x
#     return stats.norm.logpdf(a, 0, 1) + stats.norm.logpdf(b, 0, 1)

def initialise():
    return np.random.normal(0, 5, 2)

## Metropolis Sampler

Next, we create our sampler.
For the Metropolis algorithm, we also need to specify the standard deviation of the proposal distribution 
(more on this later).

In [ ]:
proposal_sd = [.25, .25]
sampler = ezmc.MetropolisSampler(func=posterior_density, 
                                 par_names=['θ1', 'θ2'], 
                                 proposal_sd=proposal_sd, init_func=initialise,)
sampler

Finally, we'll run 4 chains, with 500 steps each to begin

In [ ]:
sampler.sample_chains(n=500)

In [ ]:
chains = sampler.get_chains()
ezmc.viz.traceplot(chains)
plt.show()

It looks like the chains haven't converged yet, so let's run some more samples.
New samples are automatically appended to the end of the existing chains.

In [ ]:
sampler.sample_chains(n=2000)
chains = sampler.get_chains()
ezmc.viz.traceplot(chains)

Now that the sampler has converged, we can discard the pre-convergence samples, and thin the samples that remain to reduce autocorrelations. These are our samples from the posterior distribution.

In [ ]:
results = sampler.get_results(burn_in=2000, thin=10)
ezmc.viz.traceplot(results, pars=['θ1', 'θ2'])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
plt.sca(axes[0])
setup_axes(newfigure=False)
plt.scatter(true_posterior[0], true_posterior[1], alpha=.5)
plt.title('Direct samples')

plt.sca(axes[1])
setup_axes(newfigure=False)
plt.scatter(results['θ1'], results['θ2'], alpha=.5, color='g')
plt.title('MCMC samples')
plt.tight_layout()
plt.show()


`.get_results()` and `.get_chains()` return Pandas data frames,
so we can work with them as we please.

In [ ]:
results.head()

In [ ]:
results.mean()

In [ ]:
results.std()

Future versions of EZMC will include convenience functions for posterior summaries.

## Differential Evolution

EZMC also includes an implementation of the Differential Evolution Monte Carlo sampler (DEMC) [**references**].
This sampler can be more effective when sampling from distributions with strong correlations between parameters,
as is often the case in cognitive models, for instance.
The Metropolis algorith can be slow to sample from these distributions. 
DEMC also doesn't require tuning of the proposal SD.

Let's try to sample from a distribution with strong correlations.

In [ ]:
def sample_true(n):
    dist = stats.multivariate_normal(mean=[0, 0], cov=[[1, .9], [.9, 1]])
    return dist.rvs(n).T

def posterior_density(x):
    dist = stats.multivariate_normal(mean=[0, 0], cov=[[1, .9], [.9, 1]])
    return dist.logpdf(x)

true_posterior = sample_true(1000)
print(true_posterior.mean(1).round(2), true_posterior.std(1).round(2))

setup_axes()
plt.scatter(true_posterior[0], true_posterior[1], alpha=.5)
plt.show()

DEMC does require that we set bounds on each parameter.
The sampler initialises a large number of chains at random values between these bounds,
and gradually homes in on the high-density regions of the posterior.
Generally, these bounds should be very wide around plausible parameter values.

In [ ]:
reload(ezmc)
reload(ezmc.demc)

In [ ]:
bounds = [[-20, 20],
          [-20, 20]]
demc_sampler = ezmc.DifferentialEvolutionSampler(func=posterior_density,
                                                 par_names=['θ1', 'θ2'],
                                                 n_chains=20,
                                                 init_bounds=bounds)
# demc_sampler.sample_chains(n=2000)

In [ ]:
demc_sampler.sample_chains(5000)

In [ ]:
chains = demc_sampler.get_chains()
ezmc.viz.traceplot(chains);

In [ ]:
results = demc_sampler.get_results(burn_in=500, thin=50)
ezmc.viz.traceplot(results, pars=['θ1', 'θ2']);


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
plt.sca(axes[0])
setup_axes(newfigure=False)
plt.scatter(true_posterior[0], true_posterior[1], alpha=.5)
plt.title('Direct samples')

plt.sca(axes[1])
setup_axes(newfigure=False)
plt.scatter(results['θ1'], results['θ2'], alpha=.5, color='g')
plt.title('MCMC samples')
plt.tight_layout()
plt.show()


In [ ]:
print('Means')
print(results.mean())
print('\nSDs')
print(results.std())
print('\nCorr: %.2f' % np.corrcoef(results['θ1'], results['θ2'])[0,1])

# Cognitive Modelling

Let's work through a more realistic example.


In [ ]:
def simulate_trials(pars, n_trials=1000, max_t=10., dt=.001):
    ndt, drift, s2 = pars
    threshold = 1.
    t = np.arange(0, max_t, dt)
    signal = np.where(t > ndt, drift, 0)
    drift = np.cumsum(signal*dt)
    noise = np.random.normal(0, s2, (n_trials, len(t)))
    diffusion = np.cumsum(noise * np.sqrt(dt), 1)
    X = drift + diffusion
    return X

def X_to_rt(X, threshold=1., max_t=10., dt=.001):
    t = np.arange(0, max_t, dt)
    rt_ix = np.argmax(X > threshold, 1)
    rt = t[rt_ix]
    rt[rt_ix == 0.] = np.nan    
    return rt    

def simulate_rts(pars, n_trials=1000, max_t=10., dt=.001):
    threshold = 1.
    X = simulate_trials(pars, n_trials=n_trials, max_t=max_t, dt=dt)
    return X_to_rt(X, max_t=max_t, dt=dt)

def plot_simlations(X, rt, max_t=10., dt=.001):
    t = np.arange(0, max_t, dt)    
    fig, axes = plt.subplots(2, 1, figsize=(8, 6), gridspec_kw={'height_ratios':[.5, 1.] });

    plt.sca(axes[0])
    plt.hist(rt)
    plt.xlim(0, 5)
    plt.xticks([])
    plt.ylabel('RT distribution')

    plt.sca(axes[1])
    for i in range(len(X)):
        plt.plot(t, X[i], 'b', alpha=.1)
    plt.hlines(1., 0, 10)
    plt.xlim(0, 5)
    plt.ylim(-1, 1)
    plt.ylabel('Accumulator')
    plt.xlabel('Time (s)')

    plt.tight_layout()
    plt.show()

In [ ]:
true_pars = [1., 2., .2] # Non-decision time, Drift rate, Noise SD
t = np.arange(0, 10, .001)
true_X = simulate_trials(true_pars, n_trials=300)
true_rts =  X_to_rt(true_X)

plot_simlations(true_X, true_rts)

In [ ]:
def posterior_density(pars, true_rts=true_rts):
    ndt, drift, s2 = pars
    if ndt < 0 or s2 < 0:
        return -1e+16
    ## Normal(0, 100) priors on all parameters
    nlpdf = stats.norm.logpdf
    log_prior = nlpdf(0, 100, ndt) + nlpdf(0, 100, drift) + nlpdf(0, 100, s2)
    ## Simulate from model
    sim_rt = simulate_rts(pars, n_trials=1000)
    ## Use simulated RTs to estimate liklihood of observed rts
    p_response = 1 - np.mean(np.isnan(sim_rt)) # Normalising for simulations that don't cross threshold
    if p_response < .5:
        return -1e+16
    kernel = stats.kde.gaussian_kde(sim_rt[~np.isnan(sim_rt)], bw_method='silverman')
    log_lik = np.sum(kernel.logpdf(true_rts)) + np.log(p_response)
    log_posterior = log_prior + log_lik
    # print(repr(pars), log_posterior)
    if np.isnan(log_posterior):
        return -1e+16
    return log_posterior

posterior_density(true_pars)

In [ ]:
# posterior_density([3.62982798, 4.05647564, 1.90074922])

In [ ]:
## Reload stuff
from importlib import reload
import ezmc
reload(ezmc)
reload(ezmc.base)
reload(ezmc.samplers)
reload(ezmc.utils)
import ezmc


In [ ]:
bounds = [[0, 5],
          [0, 10],
          [0, 2]]
sampler = ezmc.DifferentialEvolutionSampler(func=posterior_density,
                                                 par_names=['ndt', 'drift', 'noise'],
                                                 init_bounds=bounds)



In [ ]:
sampler.sample_chains(nchains=10, n=1)

In [ ]:
chains = sampler.get_chains()
traceplot(chains, ['ll'])
plt.ylim(-4000, 1)

In [ ]:
chains[chains['chain']==2]['ll'].plot()

In [ ]:
sampler.sample_chains(nchains=10, n=200)

In [ ]:



chains = demc_sampler.get_chains()
ezmc.viz.traceplot(chains)

results = demc_sampler.get_results(burn_in=500, thin=50)
ezmc.viz.traceplot(results, pars=['θ1', 'θ2'])


fig, axes = plt.subplots(1, 2, figsize=(8, 4))
plt.sca(axes[0])
setup_axes(newfigure=False)
plt.scatter(true_posterior[0], true_posterior[1], alpha=.5)
plt.title('Direct samples')

plt.sca(axes[1])
setup_axes(newfigure=False)
plt.scatter(results['θ1'], results['θ2'], alpha=.5, color='g')
plt.title('MCMC samples')
plt.tight_layout()
plt.show()


print('Means')
print(results.mean())
print('\nSDs')
print(results.std())
print('\nCorr: %.2f' % np.corrcoef(results['θ1'], results['θ2'])[0,1])

In [ ]:
true_posterior = sample_true(1000).T
plt.scatter(true_posterior[0], true_posterior[1])

# DEMC

In [ ]:
# ezmc.DifferentialEvolutionSampler?

In [ ]:
m = ezmc.DifferentialEvolutionSampler(func=f, 
                                      par_names=['a', 'b'],
                                     init_bounds=[[-10, 10],
                                                 [-10, 10]])

In [ ]:
m.sample_chains(nchains=10, n=20000, verbose=1)

In [ ]:
chains = m.get_chains()
ezmc.viz.traceplot(chains)

In [ ]:
results = m.get_results(burn_in=500, thin=20)
ezmc.viz.traceplot(results, ['a', 'b'])

In [ ]:
plt.scatter(results['a'], results['b'])

In [ ]:
print(np.mean(true_posterior, 1), np.std(true_posterior, 1), np.corrcoef(true_posterior)[0,1])

In [ ]:
a = results[['a', 'b']].values.T
print(np.mean(a, 1), np.std(a, 1), np.corrcoef(a)[0,1])

In [ ]:
def kde_countour(x, y, colors='k'):
    deltaX = (max(x) - min(x))/10
    deltaY = (max(y) - min(y))/10
    xmin = min(x) - deltaX
    xmax = max(x) + deltaX
    ymin = min(y) - deltaY
    ymax = max(y) + deltaY
    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = stats.gaussian_kde(values)
    f = np.reshape(kernel(positions).T, xx.shape)
    cset = plt.contour(xx, yy, f, colors=colors)
    
kde_countour(results['a'], results['b'], colors='red')
kde_countour(p[0], p[1], colors='blue')

In [ ]:
np.sin(p[0, 0], p[1, 0])

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(results['a']);

In [ ]:
ezmc.viz.traceplot(results, ['ll'])

In [ ]:
raise Exception

# Multiple Chains

In [ ]:
m = ezmc.MetropolisSampler(func=f, par_names=['a', 'b'], proposal_sd=[.25, .25], init_func=init_func)
m.sample_chains(4, n=3000, verbose=0)

In [ ]:
chains = m.get_chains()

In [ ]:
ezmc.viz.traceplot(chains, ['a', 'b'])

In [ ]:
results = m.get_results(burn_in=500, thin=2)

In [ ]:
traceplot(results)

In [ ]:
results.mean()

In [ ]:
results.std()

# One Chain

In [ ]:
m = ezmc.MetropolisSampler(func=f, par_names=['a', 'b'], proposal_sd=[.25, .25], init_func=init_func)
m.add_chains(1)

In [ ]:
m.sample_chain(chain_ix=0, n=100)

In [ ]:
chains = m.chains[0].get_results(burn_in=0, thin=1)
chains[['a', 'b']].plot()

In [ ]:
results = m.chains[0].get_results(burn_in=2000, thin=10)
results.head()
results[['a', 'b']].hist(bins=10)

In [ ]:
print(results.mean())
print(results.std())

In [ ]:
plt.scatter(results['a'], results['ll'])

In [ ]:
plt.scatter(results['b'], results['ll'])

In [ ]:
stats.pearsonr(results['a'], results['b'])

In [ ]:
plt.scatter(results['a'], results['b'])

In [ ]:
chains['ll'].plot()

In [ ]:
m = ezmc.MetropolisSampler

In [ ]:
chain.chain